In [6]:
import pandas as pd
import requests

class ScriptData:
    def __init__(self, api_key):
        self.api_key = api_key
    
    def fetch_intraday_data(self, script):
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=1min&apikey={self.api_key}"
        response = requests.get(url)
        data = response.json()["Time Series (1min)"]
        return data
    
    def convert_intraday_data(self, script):
        data = self.fetch_intraday_data(script)
        df = pd.DataFrame.from_dict(data, orient='index')
        df.columns = ['open', 'high', 'low', 'close', 'volume']
        df.index = pd.to_datetime(df.index)
        df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].astype(float)
        df['volume'] = df['volume'].astype(int)
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'timestamp'}, inplace=True)
        return df

    
    def __getitem__(self, script):
        return self.convert_intraday_data(script)
    
    def __setitem__(self, script, value):
        # Not implemented for simplicity
        pass
    
    def __contains__(self, script):
        # Not implemented for simplicity
        pass
    
    
api_key = "EEJDQJECI4WFSVUE"  # Replace with your actual API key
script_data = ScriptData(api_key)

# Fetch and convert intraday data for a specific script
df = script_data.convert_intraday_data("GOOGL")
print(df)

# Access data for a script using getitem method
df_aapl = script_data["AAPL"]
print(df_aapl)




             timestamp      open    high     low   close  volume
0  2023-06-13 19:59:00  123.8100  123.83  123.81  123.81    1384
1  2023-06-13 19:56:00  123.8400  123.85  123.79  123.80    1509
2  2023-06-13 19:55:00  123.8300  123.84  123.80  123.84    2085
3  2023-06-13 19:54:00  123.8400  123.84  123.84  123.84    1087
4  2023-06-13 19:53:00  123.8400  123.84  123.83  123.83     873
..                 ...       ...     ...     ...     ...     ...
95 2023-06-13 16:43:00  123.7000  123.70  123.70  123.70     404
96 2023-06-13 16:40:00  123.6200  123.62  123.62  123.62     203
97 2023-06-13 16:39:00  123.6100  123.61  123.61  123.61     412
98 2023-06-13 16:38:00  123.6200  123.65  123.62  123.65    4906
99 2023-06-13 16:37:00  123.6113  123.83  123.55  123.62   25928

[100 rows x 6 columns]
             timestamp      open    high       low     close  volume
0  2023-06-13 19:59:00  183.0800  183.08  183.0500  183.0799    5192
1  2023-06-13 19:58:00  183.0800  183.08  183.0800  183.08

In [14]:
import pandas as pd

def indicator1(df, timeperiod):
    df['indicator'] = df['close'].rolling(window=timeperiod).mean()
    return df[['timestamp', 'indicator']]
# Assuming you have already fetched and converted the intraday data for a script
df = script_data.convert_intraday_data("AAPL")

# Calculate moving average with a time period of 5
ma_df = indicator1(df, timeperiod=5)
print(ma_df)



             timestamp  indicator
0  2023-06-13 19:59:00        NaN
1  2023-06-13 19:58:00        NaN
2  2023-06-13 19:57:00        NaN
3  2023-06-13 19:56:00        NaN
4  2023-06-13 19:55:00  183.08398
..                 ...        ...
95 2023-06-13 17:24:00  183.26024
96 2023-06-13 17:22:00  183.25800
97 2023-06-13 17:21:00  183.26000
98 2023-06-13 17:20:00  183.26000
99 2023-06-13 17:18:00  183.26800

[100 rows x 2 columns]
             timestamp  indicator
0  2023-06-13 19:59:00        NaN
1  2023-06-13 19:58:00        NaN
2  2023-06-13 19:57:00        NaN
3  2023-06-13 19:56:00        NaN
4  2023-06-13 19:55:00  183.08398
..                 ...        ...
95 2023-06-13 17:24:00  183.26024
96 2023-06-13 17:22:00  183.25800
97 2023-06-13 17:21:00  183.26000
98 2023-06-13 17:20:00  183.26000
99 2023-06-13 17:18:00  183.26800

[100 rows x 2 columns]


In [13]:
class Strategy:
    def __init__(self, api_key):
        self.data = ScriptData(api_key)
    
    def generate_signals(self, script):
        # Fetch intraday historical data
        df = self.data.convert_intraday_data(script)
        
        # Compute indicator data
        indicator_data = indicator1(df, timeperiod=5)
        
        # Generate signals DataFrame
        signals = pd.DataFrame({'timestamp': df['timestamp'], 'signal': 'NO_SIGNAL'})
        signals.loc[signals['timestamp'].isin(indicator_data['timestamp']), 'signal'] = 'BUY'
        signals.loc[signals['timestamp'].isin(indicator_data['timestamp']), 'signal'] = 'SELL'
        
        return signals
    
    def get_signals(self, script):
        signals = self.generate_signals(script)
        filtered_signals = signals[signals['signal'].isin(['BUY', 'SELL'])]
        print(filtered_signals)
        
api_key = "EEJDQJECI4WFSVUE"  # Replace with your actual API key
strategy = Strategy(api_key)

# Generate and print signals for a script
strategy.get_signals("AAPL")


             timestamp signal
0  2023-06-13 19:59:00   SELL
1  2023-06-13 19:58:00   SELL
2  2023-06-13 19:57:00   SELL
3  2023-06-13 19:56:00   SELL
4  2023-06-13 19:55:00   SELL
..                 ...    ...
95 2023-06-13 17:24:00   SELL
96 2023-06-13 17:22:00   SELL
97 2023-06-13 17:21:00   SELL
98 2023-06-13 17:20:00   SELL
99 2023-06-13 17:18:00   SELL

[100 rows x 2 columns]
